In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.7 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer,BertForMaskedLM,AlbertTokenizer,DistilBertTokenizer
import torch

In [ ]:
berttokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=BertForMaskedLM.from_pretrained('bert-base-uncased')
distilberttokenizer=DistilBertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.


In [ ]:
text=("I need to buy [MASK] book")

In [ ]:
input=berttokenizer(text,return_tensors='pt')
input.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
db=distilberttokenizer(text,return_tensors='pt')
distilberttokenizer.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
input.input_ids

tensor([[ 101, 1045, 2342, 2000, 4965, 1037, 2338,  102]])

In [ ]:
input['labels']=input.input_ids.detach().clone()

In [ ]:
input

{'input_ids': tensor([[ 101, 1045, 2342, 2000, 4965, 1037, 2338,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[ 101, 1045, 2342, 2000, 4965, 1037, 2338,  102]])}

In [ ]:
rand=torch.rand(input.input_ids.shape)
rand.shape

torch.Size([1, 8])

In [ ]:
rand

tensor([[0.3070, 0.3631, 0.1554, 0.7962, 0.0334, 0.7789, 0.9278, 0.1223]])

In [ ]:
mask_arr=rand<0.15*(input.input_ids !=101)*(input.input_ids !=102)
mask_arr

tensor([[False, False, False, False,  True, False, False, False]])

In [ ]:
selection=torch.flatten(mask_arr[0].nonzero()).tolist()
selection

[4]

In [ ]:
input.input_ids[0,selection]=103
input.input_ids

tensor([[ 101, 1045, 2342, 2000,  103, 1037, 2338,  102]])

In [ ]:
with torch.no_grad():
  prediction = model(input['input_ids']).logits


In [ ]:
predicted_token_id = torch.argmax(prediction[0, selection]).item()
predicted_token = tokenizer.convert_ids_to_tokens(predicted_token_id)



In [ ]:
print("Original Sentence:", text)
print("Predicted Token:", predicted_token)


Original Sentence: I need to buy a book
Predicted Token: read


In [ ]:
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.2 MB/s eta 0:00:00
